# app

rss sources:
https://tw.stock.yahoo.com/rss_index.html

In [1]:
%cd /workspace/twint/app
# !pip install -e .
# !python -m pytest tests/test_scrapers.py::test_cnbc_page_tags -v

/workspace/twint/app


In [2]:
%cd /workspace/twint/app
# !python -m app.main run.scraper=rss run.n_workers=1 run.loop_every=86400 
!python -m app.main run.scraper=cnbc run.n_workers=1

/workspace/twint/app
{'run': {'scraper': 'cnbc', 'n_workers': 1, 'loop_every': None}, 'store': {'es': {'host': 'es:9200'}, 'db': {'user': 'sqlpad', 'password': 'sqlpad', 'host': 'pg', 'dbname': 'test'}}, 'scraper': {'rss': {'fetch_rss_every_n_seconds': 172800, 'csv': {'path': './resource/rss_yahoo_us.csv'}, 'force_fetch': False}, 'cnbc': {'store': 'es'}, 'cnyes_api': {'start': [2020, 5, 5], 'until': None}}, 'proxy': {'enabled': True, 'path': './proxies.txt'}}
[2020-07-18 17:42:42,891][app.scrapers.cnbc][INFO] - scraper start running: 1 workers
[2020-07-18 17:43:21,087][app.scrapers.cnbc][INFO] - page existed, skip https://cnb.cx/2AP4ulX
[2020-07-18 17:43:21,108][app.scrapers.cnbc][INFO] - page existed, skip http://cnb.cx/2FDIr2n
[2020-07-18 17:43:21,134][app.scrapers.cnbc][INFO] - page existed, skip https://cnb.cx/31VI6TJ
[2020-07-18 17:43:21,163][app.scrapers.cnbc][INFO] - page existed, skip https://cnb.cx/2ozZtv2
[2020-07-18 17:43:21,176][app.scrapers.cnbc][INFO] - page existed, skip

# Twint

In [1]:
# %cd /workspace/twint
# !pip install e .
# !twint -u CNBC
# !pip install -U fake-useragent

Requirement already up-to-date: fake-useragent in /usr/local/lib/python3.7/site-packages (0.1.11)
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [17]:
import nest_asyncio
nest_asyncio.apply()
import twint

c = twint.Config()
c.Username = "CNBC"
c.Elasticsearch = "http://es:9200"
c.Until='2015-01-01 00:00:00'

# c.Search = "fruit"
twint.run.Search(c)

# elasticsearch

query twint
```json
{
  "_source": [
    "date",
    "username"
  ],
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "username": "business"
          }
        },
        {
          "range": {
            "date": {
              "gt": "2004-01-01 00:00:00",
              "lt": "2023-01-01 00:00:00"
            }
          }
        }
      ]
    }
  },
  "from": 0,
  "size": 1000,
  "sort": [
    {
      "date": "asc"
    }
  ]
}
```

query cnyes
http://localhost:9200/news_page/_search
```json
{
  "query": {
    "bool": {
      "filter": [
        {
          "wildcard": {
            "from_url": "*cnyes.com*"
          }
        },
        {
          "range": {
            "entry_published_at": {
              "gte": "2020-05-01T00:00:00",
              "lt": "2021-01-01T00:00:00"
            }
          }
        }
      ]
    }
  },
  "from": 0,
  "size": 1000,
  "sort": [
    {
      "entry_published_at": "desc"
    }
  ]
}
```

```json
{
  "query": {
    "bool": {
      "filter": [
        {
          "wildcard": {
            "resolved_url": "*cnbc*"
          }
        }
      ]
    }
  },
  "from": 0,
  "size": 1000,
  "sort": [
    {
      "entry_published_at": "desc"
    }
  ]
}
```

# Elasticsearch Dump

https://github.com/taskrabbit/elasticsearch-dump  

```
npm install elasticdump -g
```

dump 

```
multielasticdump \
  --direction=dump \
  --match='^.*$' \
  --input=http://es:9200 \
  --output=./dump \
  --fsCompress

multielasticdump \
  --direction=load \
  --match='^.*$' \
  --input=./dump \
  --output=http://es:9200 \
  --fsCompress


elasticdump \
  --input=http://es:9200/twinttweets \
  --output=./twinttweets_mapping_20200503.json \
  --type=mapping
elasticdump \
  --input=http://es:9200/twinttweets \
  --output=./twinttweets_index_20200503.json \
  --type=data

elasticdump \
  --input=http://es:9200/twinttweets \
  --output=$ \
  | gzip > ./twinttweets_index_20200504.json.gz
  
elasticdump \
  --input=http://es:9200/news_page \
  --output=$ \
  | gzip > ./news_page_index_20200615.json.gz
```

import

```
elasticdump \
  --input=./twinttweets_index_20200602.json.gz \
  --output=http://es:9200/twinttweets \
  --fsCompress
```

# Stock

https://twstock.readthedocs.io/zh_TW/latest/index.html


In [1]:
!pip install twstock

     |████████████████████████████████| 1.9 MB 853 kB/s eta 0:00:01     |█████████████████████▋          | 1.3 MB 853 kB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
